# Structural Analysis of RosettaScript PDB Output
The goal of this notebook is to develop a general method for correlating structural variation in RosettaScript output (i.e. RMSD) with interesting factors (DDG scores, ensemble size, trial sizes, kT).<br>
<br>
<b>Things to think about:</b>
* How much backbone variation is required/useful?
* Input starting PDB, WT output PDBs, and mutant output PDBs
* All-vs-all RMSDs for any outputs (DDG WT, DDG mutants, minimization comparison structures)
* RMSD for 8A repack sphere (repacking) and overall structure

<b>Table of Contents:</b>
* [RMSD-Calculator + X angles](#RMSD-Calculator-+-X-angles)<br>
* [X Angle Parser](#X-Angle-Parser)<br>
* [Mutant PBD Scraper](#Mutant-PBD-Scraper)<br>


## RMSD Calculator + X angles

This script uses pyRMSD to perform pairwaise RMSD calculations 1) globally, 2) within a shell defined by the find_neighbors function, and 3) for individual point mutations as defined by the read_mutations_resfile function.

In [ ]:
from Bio.PDB import *
import Bio.PDB
import os
import pyRMSD
from pyRMSD.matrixHandler import MatrixHandler
import pyRMSD.RMSDCalculator
from pyRMSD.availableCalculators import availableCalculators
import prody
import numpy as np
import scipy.spatial.distance

#From Kyle's Finalize.py
def read_mutations_resfile(filenum_dir):
    resfile = os.path.join(filenum_dir, 'mutations_repack.resfile')
    mutations = []
    with open(resfile, 'r') as f:
        post_start = False
        for line in f:
            if post_start:
                line = line.strip()
                pdb_resnum, chain, pikaa, mut_res = line.split()
                mutations.append( [pdb_resnum, chain, pikaa, mut_res] )
            elif line.startswith('start'):
                post_start = True
    return mutations

#From Kyle's Finalize.py
def find_neighbors(filenum_dir, pdb_path, neighbor_distance = 8.0):
    mutations = read_mutations_resfile(filenum_dir)
    parser = PDBParser(PERMISSIVE=1)
    open_strct = parser.get_structure('Open', pdb_path)

    # There should only be one model in PDB file
    num_models = 0
    for model in open_strct.get_models():
        num_models += 1
    assert( num_models == 1 )

    chain_list = [chain.get_id() for chain in open_strct[0].get_chains()]
    neighbors = set()
    for mutation in mutations:
        res_id, chain_id, pikaa, mut_aa = mutation
        mut_chain = str(chain_id)
        try:
            mut_pos = int( res_id )
            mut_insertion_code = ' '
        except ValueError:
            mut_pos = int( res_id[:-1] )
            mut_insertion_code = res_id[-1]

        mut_residue = open_strct[0][mut_chain][(' ', mut_pos, mut_insertion_code)]
        for chain in chain_list:
            for residue in [res.get_id() for res in open_strct[0][chain].get_residues()]:
                try:
                    # Kyle note - might be good to do something else for consistency, since not all residues have CB
                    dist = mut_residue['CB'] - open_strct[0][chain][residue]['CB']
                    if dist < neighbor_distance:
                        neighbors.add( (residue, chain) )
                except KeyError:
                    try:
                        dist = mut_residue['CA'] - open_strct[0][chain][residue]['CA']
                        if dist < neighbor_distance:
                            neighbors.add( (residue, chain) )
                    except KeyError:
                        pass

    return neighbors

#Kyle's RMSDWrapper.py (Spread out al over the place now)
def rmsd(input_pdbs, pyrmsd_calc, coordinates):
    if type(coordinates) is dict:
        rmsd_list = []
        for mutres, coord_set in coordinates.iteritems():
            rmsd_matrix = pyRMSD.matrixHandler.MatrixHandler().createMatrix(coord_set, pyrmsd_calc)
            rmsd_list.append([mutres, scipy.spatial.distance.squareform( rmsd_matrix.get_data() )])
        return rmsd_list
    else:
        rmsd_matrix = pyRMSD.matrixHandler.MatrixHandler().createMatrix(coordinates, pyrmsd_calc)
        return [scipy.spatial.distance.squareform( rmsd_matrix.get_data() )]
    
def global_ca_rms(input_pdbs):
    coordinates = np.array( [prody.parsePDB(input_pdb).select('calpha').getCoords() for input_pdb in input_pdbs] )
    return coordinates    
    
def neighborhood_rms(neighbors, reference, input_pdbs):
    temp = []

    for input_pdb in input_pdbs:
        atom_list = []
        hv = prody.parsePDB(input_pdb).getHierView()
        res_list = sorted( [hv[neighbor[1], neighbor[0][1]] for neighbor in neighbors] )
        for res in res_list:
            for atom in res:
                if atom.getElement() != 'H':
                    print atom
                    atom_list.append(atom.getCoords())
                else:
                    pass
        temp.append(atom_list)
        
    coordinates = np.asarray(temp)
    return coordinates
        
#Calculates sidechain rmsd        
def mutant_rms(datadir, input_pdbs):
    mutations = read_mutations_resfile(datadir)
    mutation_dict = {}
    
    for mutation in mutations:
        temp = []
        temp_nparray = []
        
        for input_pdb in input_pdbs: 
            atom_list = []
            hv = prody.parsePDB(input_pdb).getHierView()
            res_list = [hv[mutation[1], int(mutation[0])]]
            for res in res_list:
                for atom in res:
                    if atom.getElement() != 'H':
                        atom_list.append(atom.getCoords())
                    else:
                        pass
            temp.append(atom_list)
        
        temp_nparray = np.asarray(temp)
        
        mutation_dict['%s%s' %(mutation[1], mutation[0])] = temp_nparray
    print mutation_dict.keys()
    return mutation_dict

def chi_angles(datadir, input_pdbs):
    mutations = read_mutations_resfile(datadir)
    
    #http://www.ccp14.ac.uk/ccp/web-mirrors/garlic/garlic/commands/dihedrals.html
    chi1_dict = {'ARG':['N','CA','CB','CG'],
                 'ASN':['N','CA','CB','CG'],
                 'ASP':['N','CA','CB','CG'],
                 'CYS':['N','CA','CB','SG'],
                 'GLN':['N','CA','CB','CG'],
                 'GLU':['N','CA','CB','CG'],
                 'HIS':['N','CA','CB','CG'],
                 'ILE':['N','CA','CB','CG1'],
                 'LEU':['N','CA','CB','CG'],
                 'LYS':['N','CA','CB','CG'],
                 'MET':['N','CA','CB','CG'],
                 'PHE':['N','CA','CB','CG'],
                 'PRO':['N','CA','CB','CG'],
                 'SER':['N','CA','CB','OG'],
                 'THR':['N','CA','CB','OG1'],
                 'TRP':['N','CA','CB','CG'],
                 'TYR':['N','CA','CB','CG'],
                 'VAL':['N','CA','CB','CG1']}
    chi2_dict = {'ARG':['CA','CB','CG','CD'],
                 'ASN':['CA','CB','CG','OD1'],
                 'ASP':['CA','CB','CG','OD1'],
                 'GLN':['CA','CB','CG','CD'],
                 'GLU':['CA','CB','CG','CD'],
                 'HIS':['CA','CB','CG','ND1'],
                 'ILE':['CA','CB','CG1','CD'],
                 'LEU':['CA','CB','CG','CD1'],
                 'LYS':['CA','CB','CG','CD'],
                 'MET':['CA','CB','CG','SD'],
                 'PHE':['CA','CB','CG','CD1'],
                 'PRO':['CA','CB','CG','CD'],
                 'TRP':['CA','CB','CG','CD1'],
                 'TYR':['CA','CB','CG','CD1']}
    
    xangles_dict = {}
    
    for mutation in mutations:
        if mutation[3] == 'A' or 'G':
            continue
        else:
            xangles_dict['%s%s' %(mutation[0], mutation[1])] = {}
            counter = 1
            for input_pdb in input_pdbs:
                p = prody.parsePDB(input_pdb)
                hv = p.getHierView()

                current_res = hv[mutation[1], int(mutation[0])]
                templist = []

                if current_res.getResname() == 'ALA' or 'GLY':
                    continue

                else:
                    if current_res.getResname() in chi1_dict.keys():
                        current_res.select('name %s' %chi1_dict[current_res.getResname()])
                        chi1 = calcDihedral(current_res.select('name %s' %chi1_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][3]))
                        templist.append(chi1[0])

                    if current_res.getResname() in chi2_dict.keys():
                        chi2 = calcDihedral(current_res.select('name %s' %chi2_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][3]))
                        templist.append(chi2[0])

                    xangles_dict['%s%s' %(mutation[0], mutation[1])]['%s' %counter] = sorted(templist)

                counter = counter + 1

            print xangles_dict
    return xangles_dict

#Action!!!
def main():
    os.chdir('/home/james.lucas/Rotation/DDGBenchmarks_Test/')
    
    #Define things
    datadir = 'TestJobs/data/59648/'
    reference = 'TestJobs/data/59648/1TM1_EI.pdb'
    outputdir = 'TestJobs/output/59648/'
    
    # Use CUDA for GPU calculations, if avialable
    if 'QCP_CUDA_MEM_CALCULATOR' in availableCalculators():
        pyrmsd_calc = 'QCP_CUDA_MEM_CALCULATOR'
    else:
        pyrmsd_calc = 'QCP_SERIAL_CALCULATOR'
    
    input_temp = []
    for i in os.listdir(outputdir):
        if i.endswith('.pdb'):
            input_temp.append(os.path.join(outputdir, i ))
    input_pdbs = sorted(input_temp)
    
    neighbors = find_neighbors(datadir, reference, 8)
    
    point_mutants = mutant_rms(datadir, input_pdbs)
    #neighborhood = neighborhood_rms(neighbors, reference, input_pdbs)
    #global_ca = global_ca_rms(input_pdbs)
    
    asdf = rmsd(input_pdbs, pyrmsd_calc, point_mutants)
    
    for i in asdf:
        print i
        
main()

## X Angle Parser

Calculates X1 and X2 angles for each point mutation in the Zemu dataset DDG Rosettascript protocol output. The chi_angles function has been added to the RMSD calculator, this separate cell was added so that I could work on this on my Macbook... I can't figure out how to get pyRMSD on my Macbook :/

In [ ]:
import os
import json
import prody
from Bio.PDB import *

#From Kyle's Finalize.py
def read_mutations_resfile(filenum_dir):
    resfile = os.path.join(filenum_dir, 'mutations_repack.resfile')
    mutations = []
    with open(resfile, 'r') as f:
        post_start = False
        for line in f:
            if post_start:
                line = line.strip()
                pdb_resnum, chain, pikaa, mut_res = line.split()
                mutations.append( [pdb_resnum, chain, pikaa, mut_res] )
            elif line.startswith('start'):
                post_start = True
    return mutations

#From Kyle's Finalize.py
def find_neighbors(filenum_dir, pdb_path, neighbor_distance = 8.0):
    mutations = read_mutations_resfile(filenum_dir)
    parser = Bio.PDB.PDBParser(PERMISSIVE=1)
    open_strct = parser.get_structure('Open', pdb_path)

    # There should only be one model in PDB file
    num_models = 0
    for model in open_strct.get_models():
        num_models += 1
    assert( num_models == 1 )

    chain_list = [chain.get_id() for chain in open_strct[0].get_chains()]
    neighbors = set()
    for mutation in mutations:
        res_id, chain_id, pikaa, mut_aa = mutation
        mut_chain = str(chain_id)
        try:
            mut_pos = int( res_id )
            mut_insertion_code = ' '
        except ValueError:
            mut_pos = int( res_id[:-1] )
            mut_insertion_code = res_id[-1]

        mut_residue = open_strct[0][mut_chain][(' ', mut_pos, mut_insertion_code)]
        for chain in chain_list:
            for residue in [res.get_id() for res in open_strct[0][chain].get_residues()]:
                try:
                    # Kyle note - might be good to do something else for consistency, since not all residues have CB
                    dist = mut_residue['CB'] - open_strct[0][chain][residue]['CB']
                    if dist < neighbor_distance:
                        neighbors.add( (residue, chain) )
                except KeyError:
                    try:
                        dist = mut_residue['CA'] - open_strct[0][chain][residue]['CA']
                        if dist < neighbor_distance:
                            neighbors.add( (residue, chain) )
                    except KeyError:
                        pass

    return neighbors

def chi_angles(datadir, input_pdbs):
    mutations = read_mutations_resfile(datadir)
    
    #http://www.ccp14.ac.uk/ccp/web-mirrors/garlic/garlic/commands/dihedrals.html
    chi1_dict = {'ARG':['N','CA','CB','CG'],
                 'ASN':['N','CA','CB','CG'],
                 'ASP':['N','CA','CB','CG'],
                 'CYS':['N','CA','CB','SG'],
                 'GLN':['N','CA','CB','CG'],
                 'GLU':['N','CA','CB','CG'],
                 'HIS':['N','CA','CB','CG'],
                 'ILE':['N','CA','CB','CG1'],
                 'LEU':['N','CA','CB','CG'],
                 'LYS':['N','CA','CB','CG'],
                 'MET':['N','CA','CB','CG'],
                 'PHE':['N','CA','CB','CG'],
                 'PRO':['N','CA','CB','CG'],
                 'SER':['N','CA','CB','OG'],
                 'THR':['N','CA','CB','OG1'],
                 'TRP':['N','CA','CB','CG'],
                 'TYR':['N','CA','CB','CG'],
                 'VAL':['N','CA','CB','CG1']}
    chi2_dict = {'ARG':['CA','CB','CG','CD'],
                 'ASN':['CA','CB','CG','OD1'],
                 'ASP':['CA','CB','CG','OD1'],
                 'GLN':['CA','CB','CG','CD'],
                 'GLU':['CA','CB','CG','CD'],
                 'HIS':['CA','CB','CG','ND1'],
                 'ILE':['CA','CB','CG1','CD'],
                 'LEU':['CA','CB','CG','CD1'],
                 'LYS':['CA','CB','CG','CD'],
                 'MET':['CA','CB','CG','SD'],
                 'PHE':['CA','CB','CG','CD1'],
                 'PRO':['CA','CB','CG','CD'],
                 'TRP':['CA','CB','CG','CD1'],
                 'TYR':['CA','CB','CG','CD1']}
    
    xangles_dict = {}
    
    for mutation in mutations:
        if mutation[3] == 'A' or 'G':
            continue
        else:
            xangles_dict['%s%s' %(mutation[0], mutation[1])] = {}
            counter = 1
            for input_pdb in input_pdbs:
                p = prody.parsePDB(input_pdb)
                hv = p.getHierView()

                current_res = hv[mutation[1], int(mutation[0])]
                templist = []

                if current_res.getResname() == 'ALA' or 'GLY':
                    continue

                else:
                    if current_res.getResname() in chi1_dict.keys():
                        current_res.select('name %s' %chi1_dict[current_res.getResname()])
                        chi1 = calcDihedral(current_res.select('name %s' %chi1_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi1_dict[current_res.getResname()][3]))
                        templist.append(chi1[0])

                    if current_res.getResname() in chi2_dict.keys():
                        chi2 = calcDihedral(current_res.select('name %s' %chi2_dict[current_res.getResname()][0]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][1]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][2]),
                                            current_res.select('name %s' %chi2_dict[current_res.getResname()][3]))
                        templist.append(chi2[0])

                    xangles_dict['%s%s' %(mutation[0], mutation[1])]['%s' %counter] = sorted(templist)

                counter = counter + 1

            print xangles_dict

def main():
    os.chdir('/home/james.lucas/Rotation/DDGBenchmarks_Test/')
    #os.chdir('/Users/jameslucas/Kortemme_Rotation')
    #Define things
    datadir = 'TestJobs/data/59648/'
    reference = 'TestJobs/data/59648/1TM1_EI.pdb'
    outputdir = 'TestJobs/output/59648/'
    
    input_temp = []
    for i in os.listdir(outputdir):
        if i.endswith('.pdb'):
            input_temp.append(os.path.join(outputdir, i ))
    input_pdbs = sorted(input_temp)
    
    neighbors = find_neighbors(datadir, reference, 8)
    
    chi_angles(datadir, input_pdbs)
    
main()

## Mutant PBD Scraper
This script will take the sequence for each PDB in a subdirectory within some_ZeMu_dataset/data/ and modify the sequence based on the associated resfile. The script will then use prody_blast to find and download relevant structures from the PDB.

In [ ]:
import os
import prody
import re
import requests
import klab.bio.rcsb as ASDF

#From Kyle's Finalize.py
def read_mutations_resfile(filenum_dir):
    resfile = os.path.join(filenum_dir, 'mutations_repack.resfile')
    mutations = []
    with open(resfile, 'r') as f:
        post_start = False
        for line in f:
            if post_start:
                line = line.strip()
                pdb_resnum, chain, pikaa, mut_res = line.split()
                mutations.append( [pdb_resnum, chain, pikaa, mut_res] )
            elif line.startswith('start'):
                post_start = True
    return mutations

def generate_mut_seq(pdbdir, hv):
    blast_dict = {}
    mutation_dict = {}
        
    mutations = read_mutations_resfile(pdbdir)
        
    for mutation in mutations:
        mutation_dict[mutation[0] + mutation[1]] = mutation[3]
    
    #Dictionary: 3- to 1-letter code
    res_dict = {
        'ALA':'A',
        'CYS':'C',
        'ASP':'D',
        'GLU':'E',
        'PHE':'F',
        'GLY':'G',
        'HIS':'H',
        'ILE':'I',
        'LYS':'K',
        'LEU':'L',
        'MET':'M',
        'ASN':'N',
        'PRO':'P',
        'GLN':'Q',
        'ARG':'R',
        'SER':'S',
        'THR':'T',
        'VAL':'V',
        'TRP':'W',
        'TYR':'Y'
    }
    
    for chain in hv:
        chain_string = ''
        for residue in chain:
            current_res =  str(residue.getResnum()) + str(residue.getChid())
            if current_res not in mutation_dict.keys():
                chain_string = chain_string + res_dict[residue.getResname()]
            else:
                chain_string = chain_string + mutation_dict[str(residue.getResnum()) + str(residue.getChid())]
        blast_dict[chain.getChid()] = chain_string
    
    for i in blast_dict:
        print blast_dict[i]
        
    return blast_dict

def get_PDBs(pdbdir):
    skip_blast = False
    
    print pdbdir
    
    for pdb_file in os.listdir(pdbdir):
        if pdb_file.endswith('.pdb'):
            output_dir = os.path.join('/kortemmelab/home/james.lucas/Mutant_PDBs', pdb_file[:-4])
            try:
                os.makedirs(output_dir)        
            except:
                print 'File directory %s already exists!!!' % pdb_file[:-4]
                #skip_blast = True
            
            pdb_chains = re.split(r'_', pdb_file)[1][:-4]
            pdb_id = re.split(r'_', pdb_file)[0]
            hv = prody.parsePDB(os.path.join(pdbdir, pdb_file)).getHierView()
            
    if skip_blast == True:
        skip_blast = False
        
    else:
        blast_dict = generate_mut_seq(pdbdir, hv)
        hit_list = []
        for sequence in blast_dict:
            set_dict = {}
            temp_set = set()
            blast_me = prody.blastPDB(blast_dict[sequence])
            hits_dict = blast_me.getHits(percent_identity=95)
            for hit in hits_dict:
                temp_set.add(hit)
            
            hit_list = hit_list.append(temp_set)
            
        common_set = hit_list[0]
        
        for entry in hit_list[1:]:
            common_set = common_set.intersection(entry)
        
        print common_set
        
#         for hit in hits:
#             ASDF.download_pdb(hit, output_dir)
            
def main():
    os.chdir('/kortemmelab/home/james.lucas/160322-james-backrub-rscript-full/data/')
    cwd = os.getcwd()
    print cwd
    for pdbdir in os.listdir(cwd):
        if os.path.isdir(pdbdir):
            get_PDBs(pdbdir)
        break
        
main()


# for subdir in directory(data):
#   for files in subdir:
#   mutlist = []
#   if file == 'mutations.resfile':
#       mut_type = mutant residue type
#       mut_pos = mutant residue position
#       mutlist.append([mut_type, mut_pos])
#   if file ends in '.pdb':
#       structure = parser.get_structure('TEST', data + subdir + files)
#       pdbcode = take the first four letters of filename.pdb

#   #Find related structures using PyPDB module make_query()
#   MSQuery = pypdb.make_query(pdbcode, querytype = 'ModifiedStructuresQuery')

#   #Obtain PDB sequence, introduce mutations, and search against PDB
#   ppb=CaPPBuilder()
#   for pp in ppb.build_peptides(structure): 
#       print pp.get_sequence()
#   seq = polypeptide.get_sequence()
#   search_PDB_for_sequence(seq)

## Kyle's RMSD Script 

In [ ]:
#!/usr/bin/python

program_description = "Script to calculate pairwise RMSD of ensemble"

# Use pyRMSD for RMSD calculations if available as it's much faster than BioPython
try:
    import pyRMSD
    from pyRMSD.matrixHandler import MatrixHandler
    import pyRMSD.RMSDCalculator
    from pyRMSD.availableCalculators import availableCalculators
    # Use CUDA for GPU calculations, if avialable
    if 'QCP_CUDA_MEM_CALCULATOR' in availableCalculators():
        pyrmsd_calc = 'QCP_CUDA_MEM_CALCULATOR'
    else:
        pyrmsd_calc = 'QCP_SERIAL_CALCULATOR'
except ImportError:
    print 'Warning: could not import faster RMSD module. Falling back to slower biopython...'
    pyRMSD = None
    import Bio.PDB
    import Bio.PDB.Atom as Atom

def calc_rms(ref_coords, alt_coords):
    # print ref_coords, alt_coords
    assert( len(ref_coords) == len(alt_coords) )
    if pyRMSD:
        calculator = pyRMSD.RMSDCalculator.RMSDCalculator(pyrmsd_calc, np.array([ref_coords, alt_coords]))
        return calculator.pairwiseRMSDMatrix()[0]
    else:
        super_imposer = Bio.PDB.Superimposer()
        ref_atoms = [Atom.Atom('CA', coords, 0.0, 1.0, '', ' CA ', i+1, element='C') for i, coords in enumerate(ref_coords)]
        alt_atoms = [Atom.Atom('CA', coords, 0.0, 1.0, '', ' CA ', i+1, element='C') for i, coords in enumerate(alt_coords)]
        super_imposer.set_atoms(ref_atoms, alt_atoms)
        return super_imposer.rms

def main():
    parser = argparse.ArgumentParser(description=program_description)

    parser.add_argument('-d', '--database_input',
                        nargs = '+',
                        help = 'Database to read coordinates from')
    parser.add_argument('-f', '--folder',
                        nargs = '+',
                        help = 'Folder to search for PDB files')
    parser.add_argument('-p', '--pdb_file',
                        nargs = '+',
                        help = 'PDB file to cluster')
    parser.add_argument('--single_thread',
                        default = False,
                        action = 'store_true',
                        help = 'Do not use multiprocessing')

    args = parser.parse_args()

    if args.single_thread:
        cpu_count = 1
    else:
        cpu_count = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(cpu_count)

    results_dict = {}

    print 'Starting calculating RMS for all paths vs. all first fragments for each position'
    total_count = len(path_data)
    starting_time = time.time()

    def helper_callback(outer_results):
        for results_tuple in outer_results:
            path_number, rms_results = results_tuple
            results_dict[path_number] = rms_results

    path_nums_for_jobs = [[] for x in xrange(cpu_count)]
    path_coords_for_jobs = [[] for x in xrange(cpu_count)]
    for i, path_coords in enumerate([[anchor_points[x] for x in path] for path in path_data]):
        path_nums_for_jobs[ i%cpu_count ].append( i )
        path_coords_for_jobs[ i%cpu_count ].append( path_coords )

    for path_nums_list, path_coords_list in zip(path_nums_for_jobs, path_coords_for_jobs):
        if args.single_thread:
            helper_callback( paths_against_all_fragments(path_nums_list, path_coords_list, starting_time, total_count, path_length) )
        else:
            pool.apply_async(paths_against_all_fragments, (path_nums_list, path_coords_list, starting_time, total_count, path_length), callback=helper_callback)

    if not args.single_thread:
        pool.close()
        pool.join()

    n = int(reporter_n.value)
    completion_time = time.time()
    print 'Finished! Processed %d %s, took %.3f seconds\n' % (n, 'paths', completion_time-starting_time)

if __name__ == "__main__":
    main()